# Chapter 12: Advanced pandas

## 12.1 Categorical Data

### 12.1.1 Background and Motivation

`unique` and `value_counts` enable us to extract the distinct values from an array and comput their frequencies.

In [1]:
import numpy as np; import pandas as pd

In [3]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [7]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [8]:
dim

0     apple
1    orange
dtype: object

In [9]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

**Categorical** representation - This representation as integers.

**Categories** - The array of distinct values.

**Category codes** - The integer values that reference the categories.

### 12.1.2 Categorical Type in pandas

In [10]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                    'basket_id': np.arange(N),
                    'count': np.random.randint(3, 15, size=N),
                    'weight': np.random.uniform(0, 4, size=N)},
                    columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,4,2.435154
1,1,orange,9,0.357263
2,2,apple,14,0.900945
3,3,apple,3,3.269101
4,4,apple,8,1.112761
5,5,orange,10,3.519473
6,6,apple,4,0.508921
7,7,apple,9,2.946574


In [11]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [12]:
# Values of fruit_cat are not NumPy array

c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [13]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [14]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [15]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [16]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [17]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Unless explicitly specified, categorical conversion assume no specific ordering of the categories.

In [18]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

The output `['foo' < 'bar' < 'baz']` indicates that `'foo'` precedes `'bar'` in the ordering and so on.

In [19]:
# Unordered to ordered
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 12.1.3 Computations with Categoricals

In [20]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [21]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [22]:
# Add quartile names instead of exact quartiles
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [23]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [24]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
            .groupby(bins)
            .agg(['count', 'min', 'max'])
            .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [25]:
# Retains the original categorical information, including ordering, from bins
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### Better performance with categoricals

If you do a lot of analytics on a particular dataset, converting to categorical can yield substantial overall performance gains. A categorical version of a DataFrame column uses significantly less memory too.

Consider some Series with 10 million elements and a small number of distinct categories.

In [26]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N //4))

# Convert labels to categorical
categories = labels.astype('category')

labels.memory_usage()

80000128

In [27]:
categories.memory_usage()

10000332

In [30]:
# Not free but is a one-time cost
%time _ = labels.astype('category')

Wall time: 863 ms


### 12.1.4 Categorical Methods

In [31]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [32]:
# .cat provides access to categorical methods
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [33]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [34]:
# set_categories to change beyond the four values
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [35]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [36]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [37]:
# remove_unused_categories to trim unobserved categories
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [38]:
# Removes categories c and d
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

#### Creating dummy variables for modeling

You'll often transform categorical data into *dummy variables*, also called **one-hot encoding**.

This involves creating a DataFrame with a column for each distinct category; these columns contain 1s for occurrences of a given category and 0 otherwise.

In [39]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

# pandas.get_dummies converts 1-D categorical data
# Into DataFrame containing dummy variable
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 Advanced GroupBy Use

### 12.2.1 Group Transforms and "Unwrapped" GroupBys

`transform` is similar to `apply` but imposes more constraints:
- It can produce a scalar value to be broadcast to the shape of the group
- It can produce an object of the same shape as the input group
- It must not mutate its input

In [2]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [3]:
# Group means by key
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

Suppose we wanted to produce a Series of the same shape as `df['value']` but with values replaced by the average grouped by `'key'`.

In [4]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [5]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [6]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [7]:
# Compute the ranks in descending order for each group
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [8]:
# Consider a group transformation function
# Composed from simple aggregations
def normalize(x):
    return (x - x.mean()) / x.std()

In [9]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [10]:
# Equivalent to .transform()
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Built-in aggregate functions like `'mean'` and `'sum'` are much faster than a general `apply` function. They also have a "fast past" when used with `transform`, allowing **unwrapped** group operations.

In [11]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [13]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

While an unwrapped group operation may involve multiple group aggregations, the overall benefit of vectorized operations often outweigh this.

### 12.2.2 Grouped Time Resampling

For time series data, the `resample` method is semantically a group operation based on a time intervalization.

In [14]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [15]:
# Index by 'time' and resample
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


Suppose a DataFrame contains multiple time series, marked by an additional group key column.

In [16]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [27]:
# NOTE: TimeGrouper() is deprecated
# NOTE: Use pandas.Grouper()

resampled = df2.set_index('time').groupby([pd.Grouper(freq='5T'), 'key']).sum()
resampled

value
time                key       
2017-05-20 00:00:00 a     30.0
                    b     35.0
                    c     40.0
2017-05-20 00:05:00 a    105.0
                    b    110.0
                    c    115.0
2017-05-20 00:10:00 a    180.0
                    b    185.0
                    c    190.0

In [28]:
resampled.reset_index()

,time,key,value
0,2017-05-20 00:00:00,a,30.0
1,2017-05-20 00:00:00,b,35.0
2,2017-05-20 00:00:00,c,40.0
3,2017-05-20 00:05:00,a,105.0
4,2017-05-20 00:05:00,b,110.0
5,2017-05-20 00:05:00,c,115.0
6,2017-05-20 00:10:00,a,180.0
7,2017-05-20 00:10:00,b,185.0
8,2017-05-20 00:10:00,c,190.0


## 12.3 Techniques for Method Cleaning

When applying a sequence of transformations, you may find yourself creating numerous temporary variables that are never used in your analysis.

    df = load_data()
    df2 = df[df['col2'] < 0]
    df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
    result = df2.groupby('key').col1_demeaned.std()

    # Usual non-functional way
    df2 = df.copy()
    df2['k'] = v

    # Functional assign way
    df2 = df.assign(k=v)

Assigning in-place may execute faster than using `assign`, but `assign` enables easier method chaining.

    result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
              .groupby('key')
              .col1_demeaned.std())  
> Outer parentheses to make it more convenient to add line breaks

**Callables** - Function-like arguments to help with referring to the result of `load_data` before being assigned to the temporary variable `df`.

    df = load_data()
    df2 = df[df['col2] < 0]

    # Can be rewritten as
    df = (load_data()
          [lambda x: x['col2'] < 0])
    
    # Write the entire sequence as a single-chained expression
    result = (load_data()
              [lambda x: x.col2 < 0]
              .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
              .groupby('key')
              .col1_demeaned.std())

### 12.3.1 The pipe Method

    a = f(df, arg1=v1)
    b = g(a, v2, arg3=v3)
    c = h(b, arg4=v4)

    # Can be rewritten using calls to `pipe`
    result = (df.pipe(f, arg1=v1)
              .pipe(g, v2, arg3=v3)
              .pipe(h, arg4=v4))
    
The statement `f(df)` and `df.pipe(f)` are equivalent, but `pipe` makes chained invocation easier.

A useful pattern for `pipe` is to generalize sequences of operations into reusable functions.

Let's consider subtracting group means from a column.

    g = df.groupby(['key1', 'key2'])
    df['col1'] = df['col1'] g.transform('mean')

Suppose you wanted to demean more than 1 column and easily change the group keys. You might want to perform this in a method chain.

    def group_demean(df, by, cols):
        result = df.copy()
        g = df.groupby(by)
        for c in cols:
            result[c] = df[c] - g[c].transform('mean')
        return result
    
    result = (df[df.col1 < 0]
              .pipe(group_demean, ['key1', 'key2'], ['col1]))